In [ ]:
from Utils.Data_Processing import *

import os

import numpy as np
import matplotlib.pyplot as plt

In [ ]:
T_PAST = 60
T_FUT = 20

In [ ]:
DATA_DIR = './Data'
STOCK_DATA_DIR = os.path.join(DATA_DIR, 'stocks')
WINDOW_DATA_DIR = os.path.join(DATA_DIR, f'windowed_data_{T_PAST}_{T_FUT}')
WINDOW_DT_DATA_DIR = os.path.join(DATA_DIR, f'windowed_dt_data_{T_PAST}_{T_FUT}')

TEST_STOCKS = {'NVDA', 'GM', 'LMT', 'HPQ', 'FWONK', 'MSI', 'ARM', 'MSFT', 'JNJ'}

In [ ]:
train_mat = np.array([], dtype=np.float32).reshape(0, T_PAST+T_FUT)
train_mat_dt = np.array([], dtype=np.float16).reshape(0, T_PAST+T_FUT)
test_mat = np.array([], dtype=np.float32).reshape(0, T_PAST+T_FUT)
test_mat_dt = np.array([], dtype=np.float16).reshape(0, T_PAST+T_FUT)

for f_name in sorted(os.listdir(WINDOW_DATA_DIR)):
  f_dir = os.path.join(DATA_DIR, f_name)
  stock_ticker = f_name.split('.')[0][:-len('_windows')]
  
  is_test = stock_ticker in TEST_STOCKS
  
  stock_windows = np.load(os.path.join(WINDOW_DATA_DIR, f'{stock_ticker}_windows.npy'))
  stock_windows_dt = np.load(os.path.join(WINDOW_DT_DATA_DIR, f'{stock_ticker}_windows_dt.npy'))
  
  mat_for_stock = test_mat if is_test else train_mat
  dt_mat_for_stock = test_mat_dt if is_test else train_mat_dt
  
  mat_for_stock = np.concatenate(
    (mat_for_stock, stock_windows),
    axis=0
  )
  dt_mat_for_stock = np.concatenate(
    (dt_mat_for_stock, stock_windows_dt),
    axis=0
  )
  
  if stock_ticker in TEST_STOCKS:
    test_mat = mat_for_stock
    test_mat_dt = dt_mat_for_stock
  else:
    train_mat = mat_for_stock
    train_mat_dt = dt_mat_for_stock

In [ ]:
train_past, train_fut, train_dt_past, train_dt_fut = split_past_fut(train_mat, train_mat_dt, T_PAST)
test_past, test_fut, test_dt_past, test_dt_fut = split_past_fut(test_mat, test_mat_dt, T_PAST)

In [ ]:
for mat in (train_past, train_fut, test_past, test_fut):
  print(mat.min(), mat.max())

## Process Data

In [ ]:
scaled_tr_past, S0_tr_past = scale_by_1st_col(train_past)
scaled_tr_fut, S0_tr_fut = scale_by_1st_col(train_fut)

scaled_te_past, S0_te_past = scale_by_1st_col(test_past)
scaled_te_fut, S0_te_fut = scale_by_1st_col(test_fut)

In [ ]:
for mat in (scaled_tr_past, scaled_tr_fut, scaled_te_past, scaled_te_fut):
  print(mat.min(), mat.max())

In [ ]:
def make_PCA_train_matrix(scaled_tr_past):
  cut_scaled_tr = scaled_tr_past[:, 1:]
  col_means = np.mean(cut_scaled_tr, axis=0, keepdims=True)
  
  return col_means, cut_scaled_tr - col_means

def make_PCA_test_matrix(scaled_te_past, tr_col_means):
  return scaled_te_past[:, 1:] - tr_col_means

In [ ]:
tr_col_means, cut_scaled_tr_past = make_PCA_train_matrix(scaled_tr_past)
cut_scaled_te_past = make_PCA_test_matrix(scaled_te_past, tr_col_means)

## Compute PCA

In [ ]:
U, sigma, Vt = np.linalg.svd(cut_scaled_tr_past)

In [ ]:
energies = np.cumsum(sigma**2)/np.sum(sigma**2)
ENERGY_THRESHOLD = 0.99
num_modes = np.argmax(energies >= ENERGY_THRESHOLD)

plt.plot(energies)
plt.axhline(y=ENERGY_THRESHOLD, color='0.8', linestyle='--')
print(f'# eigenvals to explain {ENERGY_THRESHOLD}% Var:', num_modes)

In [ ]:
for i in range(5):
  plt.plot(Vt[i, :], label=f'{i+1}th PC mode')
plt.legend();

In [ ]:
plt.figure(figsize=(10,10))

plt.gca().set_prop_cycle('color', [plt.get_cmap('tab20')(i) for i in range(20)])
for i in range(num_modes):
  plt.plot(Vt[i, :], label=f'{i+1}th PC mode')
plt.legend();

In [ ]:
for i in range(num_modes, 5+num_modes):
  plt.plot(Vt[i, :], label=f'{i+1}th PC mode')
plt.legend();

## Predict the Future

### Linear Method

#### Train Loss

In [ ]:
tr_fut_col_means = scaled_tr_fut.mean(axis=0, keepdims=True)
transform_mat = Vt[:num_modes, :].T@np.diag(1/sigma[:num_modes])@U[:, :num_modes].T@(scaled_tr_fut-tr_fut_col_means)

In [ ]:
# SVD based prediction of train_fut
tr_fut_hat = cut_scaled_tr_past@transform_mat + tr_fut_col_means
print('Train MAE', np.mean(np.abs(tr_fut_hat-scaled_tr_fut)))
print('Train MAE (last point)', np.mean(np.abs(tr_fut_hat[:, -1]-scaled_tr_fut[:, -1])))

fig, ax = plt.subplots(2, 5, figsize=(15, 10))

for i in range(10):
  ax[i//5, i%5].plot(tr_fut_hat[i, :].T, label='pred. fut.')
  ax[i//5, i%5].plot(scaled_tr_fut[i, :].T, label='act. fut.')
  ax[i//5, i%5].legend()

#### Test Loss

In [ ]:
# SVD based prediction of test_fut
test_future_hat = cut_scaled_te_past@transform_mat + tr_fut_col_means
print('Test MAE', np.mean(np.abs(test_future_hat-scaled_te_fut)))
print('Test MAE (last pt)', np.mean(np.abs(test_future_hat[:, -1]-scaled_te_fut[:, -1])))

fig, ax = plt.subplots(2, 5, figsize=(20, 10))

for i in range(10):
  ax[i//5, i%5].plot(test_future_hat[i, :].T, label='pred. fut.')
  ax[i//5, i%5].plot(scaled_te_fut[i, :].T, label='act. fut.')
  ax[i//5, i%5].legend()